# Spectrogram Emotion Prototype Learning

*Code fully written by Daniel Peña Porras*.


---



*    #### **PREPROCESS** of audio samples from different speech emotion recognition datasets.


---

In [ ]:
'''
@author: Daniel Peña Porras.
'''

##### Hyperparameters

In [ ]:
prepocess_window_size_ms = 100
preprocess_window_overlapping_ms = 20

##### Dependencies

In [ ]:
! pip install librosa
! pip install mir_eval

In [ ]:
import tensorflow as tf
tf.compat.v1.enable_eager_execution()

2025-06-05 08:08:46.136297: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [ ]:
import os
import soundfile as sf
import IPython.display as ipd

import librosa
import librosa.display as dsp

import mir_eval

from IPython.display import Audio
from scipy.io import wavfile

import matplotlib.pyplot as plt
import numpy as np
import time
import warnings
warnings.filterwarnings('ignore')

import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
%matplotlib inline

import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from sklearn import preprocessing

from skimage.transform import resize
from scipy.ndimage import gaussian_laplace

from keras.callbacks import TensorBoard

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.layers import LeakyReLU
import os
from keras import backend as K
from tensorflow.keras.utils import plot_model, model_to_dot
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, Callback

from skimage.transform import resize
from keras.callbacks import Callback
from keras.applications import VGG19, EfficientNetB0, ResNet50V2, DenseNet121
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, Lambda, Concatenate
from keras.optimizers import Adam
from keras.losses import MeanSquaredError

from scipy.ndimage import gaussian_laplace

In [ ]:
from glob import glob

In [ ]:
os.environ["KERAS_BACKEND"] = "tensorflow"

## **Data preprocessing**

### Load audio samples and convert to spectrograms

#### Define path and metadata


In [ ]:
dataset = 'EmoDB' # or 'EmoMatch' 'RAVDESS'

In [ ]:
dataset_path = f'/home/jovyan/Daniel/Exp_{dataset}/datasets/raw/'
output_audio_path = f'/home/jovyan/Daniel/Exp_{dataset}/datasets/windowed_{prepocess_window_size_ms}ms_{dataset}/'

In [ ]:
if dataset == 'RAVDESS':
    sr = 48000
    n_speakers = 24
    speakers = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,
                14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
    n_phonemes = 44
    n_classes = 8
    class_names = [
        'neutral',
        'calm',
        'happy',
        'sad',
        'angry',
        'fearful',
        'disgust',
        'surprised'
    ]
    emotion_map = {
        "01": "neutral",
        "02": "calm",
        "03": "happy",
        "04": "sad",
        "05": "angry",
        "06": "fear",
        "07": "disgust",
        "08": "surprise"
    }
    get_prototype_emotion = {
        0: 'neutral',
        1: 'calm',
        2: 'joy',
        3: 'sadness',
        4: 'anger',
        5: 'fear',
        6: 'disgust',
        7: 'surprise'
    }
    male_speakers = [1, 3, 5, 7, 9, 11, 13, 15, 17, 19, 21, 23]
    female_speakers = [2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24]

elif dataset == 'EmoMatch':
    sr = 48000
    # n_speakers = 50
    speakers = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 20, 22, 23,
                25, 29, 31, 33, 35, 36, 37, 40, 41, 43, 45, 46, 49]
    n_speakers = len(speakers)
    n_phonemes = 24
    n_classes = 7
    class_names = [
        'angry',
        'disgust',
        'fearful',
        'happy',
        'neutral',
        'sad',
        'surprised'
    ]
    get_prototype_emotion = {
        0: 'anger',
        1: 'disgust',
        2: 'fear',
        3: 'joy',
        4: 'neutral',
        5: 'sadness',
        6: 'surprise'
    }
    emotion_map = {
        "0": "anger",
        "1": "disgust",
        "2": "fear",
        "3": "joy",
        "4": "neutral",
        "5": "sadness",
        "6": "surprise"
    }
    rev_emotion_map = {
        "anger": "0",
        "boredom": "1",
        "disgust": "2",
        "anxiety/fear": "3",
        "happiness": "4",
        "neutral": "5",
        "sadness": "6"
    }
    spanish_english_map = {
        "Neutro": "neutral",
        "Alegria": "joy",
        "Tristeza": "sadness",
        "Ira": "anger",
        "Miedo": "fear",
        "Asco": "disgust",
        "Sorpresa": "surprise"
    }
    male_speakers = [1, 2, 5, 9, 12, 13, 22, 23, 31, 35, 36, 37, 43]
    female_speakers = [3, 4, 6, 7, 8, 10, 11, 20, 25, 29, 33, 40, 41, 45, 46, 49]

elif dataset == 'EmoDB':
    sr = 16000
    n_speakers = 10
    speakers = [3, 8, 9, 10, 11, 12, 13, 14, 15, 16]
    n_phonemes = 45
    n_classes = 7
    class_names = [
        'angry',
        'bored',
        'disgust',
        'fearful',
        'happy',
        'neutral',
        'sad'
    ]
    get_prototype_emotion = {
        0: 'angry',
        1: 'bored',
        2: 'disgust',
        3: 'fearful',
        4: 'happy',
        5: 'neutral',
        6: 'sad'
    }
    emotion_map = {
        "0": "anger",
        "1": "boredom",
        "2": "disgust",
        "3": "anxiety/fear",
        "4": "happiness",
        "5": "neutral",
        "6": "sadness"
    }
    rev_emotion_map = {
        "anger": "0",
        "boredom": "1",
        "disgust": "2",
        "anxiety/fear": "3",
        "happiness": "4",
        "neutral": "5",
        "sadness": "6"
    }
    german_english_map = {
        "W": "anger",
        "L": "boredom",
        "E": "disgust",
        "A": "anxiety/fear",
        "F": "happiness",
        "N": "neutral",
        "T": "sadness"
    }
    male_speakers = [3, 10, 11, 12, 15]
    female_speakers = [8, 9, 13, 14, 16]

#### Display audio

In [ ]:
speaker = '03'

if dataset == 'EmoDB':
  emotion = 'F'
  statement = 'a01'
  version = 'a'
  path = dataset_path + f'{speaker}{statement}{emotion}{version}.wav'
  emotion = german_english_map.get(emotion)

elif dataset == 'RAVDESS':
  emotion = '01'
  statement = '01'
  intensity = '01'
  repetition = '01'
  path = dataset_path + f'03-01-{emotion}-{intensity}-{statement}-{repetition}-{speaker}.wav'
  emotion = emotion_map.get(emotion)

elif dataset == 'EmoMatch':
  emotion = 'Alegria'
  sentence = '02'
  path = dataset_path + f'{speaker}_{emotion}_{sentence}.WAV'

print('speaker:', speaker)
print('emotion:', emotion)
print('statement:', statement)
Audio(path, rate=sr)

#### **Cut audios** in overlapping windows of a fixed length: function

In [ ]:
segment_length_ms =  prepocess_window_size_ms
overlap_length_ms = preprocess_window_overlapping_ms

segment_length_samples = int(sr * segment_length_ms / 1000)  # Convert segment length from milliseconds to samples
overlap_length_samples = int(sr * overlap_length_ms / 1000)  # Convert overlap length from milliseconds to samples

In [ ]:
def split_audio(audio_path, segment_length_samples, overlap_length_samples):
    audio, _ = librosa.load(audio_path, sr=sr)

    segments = []

    start = 0
    while start + segment_length_samples <= len(audio):
        segment = audio[start:start + segment_length_samples]
        segments.append(np.array(segment))
        start += overlap_length_samples

    return segments

#### Mel spectograms extraction parameters

In [ ]:
# sr                    # previously defined
n_fft = 1024            # length of the FFT window
window = 'hamming'      # window function
win_length = 512        # window length
hop_length = 256        # number of samples between successive frames
n_mels = 128            # number of Mel bands
fmax = sr/2             # highest frequency (in Hz)

#### Conversion to full and windowed spectrograms and labels assignation

##### Prepare lists

In [ ]:
original_windowed_audios = []

windowed_spectrograms = []
windowed_labels = []

windowed_labels_sentence = [] # we can also store the sentence and the speaker ground truths
windowed_labels_speaker = []

In [ ]:
windowed_spectrograms_male = []
windowed_labels_male = []
windowed_spectrograms_female = []
windowed_labels_female = []

In [ ]:
# Initialize dictionaries
get_spectrogram_list = {}
get_label_list = {}

# Dynamically create lists for each speaker
for spk in speakers:
    key = f"{spk:02d}"  # ensures two-digit format, e.g., 3 → "03"
    get_spectrogram_list[key] = []
    get_label_list[key] = []

##### Convert to spectrograms

In [ ]:
total_files = len([f for f in os.listdir(dataset_path) if f.endswith(".wav")])
print('Total files:', total_files)
processed_files = 0

535


In [ ]:
for filename in os.listdir(dataset_path):
    if filename.endswith(".wav"):

        processed_files += 1
        print(f"Processing file {processed_files}/{total_files}: {filename}")

        file_path = os.path.join(dataset_path, filename)

        # Labels process
        if dataset == 'EmoDB':
          filename_parts = filename.split('.')[0]

          speaker = filename_parts[0:2]

          german_emotion = filename_parts[5]
          english_emotion = german_english_map.get(german_emotion)
          emotion_id = rev_emotion_map.get(english_emotion)

          preprocess_audio = True

        elif dataset == 'RAVDESS':
          filename_parts = filename.split('-')

          speaker = filename_parts[6].split('.')[0]

          emotion_id = filename_parts[2]
          emotion = emotion_map.get(emotion_id)

          intensity = filename_parts[3]
          if intensity == '02':
            preprocess_audio = True

        elif dataset == 'EmoMatch':
          filename_parts = filename.split('_')

          speaker = filename_parts[0]

          spanish_emotion = filename_parts[1]
          english_emotion = spanish_english_map.get(spanish_emotion)
          emotion_id = rev_emotion_map.get(english_emotion)

          preprocess_audio = True


        if preprocess_audio:
          windowed_audios = split_audio(file_path, segment_length_samples, overlap_length_samples)

          for idx, audio in enumerate(windowed_audios):
              mel_spectrogram = librosa.feature.melspectrogram(y=audio, sr=sr,
                                                              n_fft=n_fft,
                                                              win_length=win_length,
                                                              hop_length=hop_length,
                                                              n_mels=n_mels,
                                                              window=window)
              mel_spectrogram_db = librosa.power_to_db(mel_spectrogram, ref=np.max)  # Convert to decibel scale (log-magnitude)

              windowed_spectrograms.append(mel_spectrogram_db)
              windowed_labels.append(emotion_id)

              # Save the windowed audio
              audio_filename = f"{filename}_segment_{idx}.wav"
              audio_path = os.path.join(output_audio_path, audio_filename)
              sf.write(audio_path, audio, sr)

              get_spectrogram_list[speaker].append(mel_spectrogram_db)
              get_label_list[speaker].append(emotion_id)

              if int(speaker) in male_speakers:
                  windowed_spectrograms_male.append(mel_spectrogram_db)
                  windowed_labels_male.append(emotion_id)
              else:
                  windowed_spectrograms_female.append(mel_spectrogram_db)
                  windowed_labels_female.append(emotion_id)

              windowed_labels_speaker.append(speaker)

Processing file 1/535: 11a01Wc.wav
Processing file 2/535: 12b02Ad.wav
Processing file 3/535: 14a01Wc.wav
Processing file 4/535: 13a07Lb.wav
Processing file 5/535: 10a07La.wav
Processing file 6/535: 14a02Ab.wav
Processing file 7/535: 08b01Aa.wav
Processing file 8/535: 16a02Nb.wav
Processing file 9/535: 15a02Ac.wav
Processing file 10/535: 13b09Ec.wav
Processing file 11/535: 10a04Fd.wav
Processing file 12/535: 16a04Tc.wav
Processing file 13/535: 03b02Aa.wav
Processing file 14/535: 14a04Aa.wav
Processing file 15/535: 14a05Lb.wav
Processing file 16/535: 13b03Ed.wav
Processing file 17/535: 15b01Lb.wav
Processing file 18/535: 13a07Tc.wav
Processing file 19/535: 12a05Nd.wav
Processing file 20/535: 11b03Lc.wav
Processing file 21/535: 16b03Fa.wav
Processing file 22/535: 08b03Lc.wav
Processing file 23/535: 03a05Nd.wav
Processing file 24/535: 16b01Wa.wav
Processing file 25/535: 09b02Tb.wav
Processing file 26/535: 14a05Ta.wav
Processing file 27/535: 13b09Fc.wav
Processing file 28/535: 08b09Wc.wav
P

In [ ]:
print(f"Number of spectrograms WITH WINDOWS: {len(windowed_spectrograms)}")
print(f"Number of emotion labels WITH WINDOWS: {len(windowed_labels)}")
print(f"Number of speaker labels WITH WINDOWS: {len(windowed_labels_speaker)}\n")

print(f"Number of spectrograms actor 1: {len(get_spectrogram_list['01'])}")
print(f"Number of labels actor 1: {len(get_label_list['01'])}\n")

print(f"Number of spectrograms of male speakers: {len(windowed_spectrograms_male)}")
print(f"Number of labels of male speakers: {len(windowed_labels_male)}\n")

print(f"Number of spectrograms of female speakers: {len(windowed_spectrograms_female)}")
print(f"Number of labels of female speakers: {len(windowed_labels_female)}\n")

Number of spectrograms WITH WINDOWS: 142823
Number of emotion labels WITH WINDOWS: 142823
Number of speaker labels WITH WINDOWS: 142823


##### Save datasets

In [ ]:
w_spectrograms = np.array(windowed_spectrograms)
w_spectrograms_32 = w_spectrograms.astype('float32')

w_labels = np.array(windowed_labels, dtype=int)
w_labels_speaker = np.array(windowed_labels_speaker, dtype=int)

if dataset == 'EmoDB' or dataset == 'EmoMatch':
  one_hot_labels_emotions = np.eye(n_classes)[w_labels]
  one_hot_labels_speakers = np.eye(n_speakers)[w_labels_speaker]
elif dataset == 'RAVDESS':
  one_hot_labels_emotions = np.eye(n_classes)[w_labels - 1]
  one_hot_labels_speakers = np.eye(n_speakers)[w_labels_speaker - 1]

np.savez(f'/home/jovyan/Daniel/Exp_{dataset}/datasets/preprocessed/{dataset}100ms32LabeledAllSpe.npz', x_train = w_spectrograms_32, y_emo = one_hot_labels_emotions)#, y_spe = one_hot_labels_speakers)

In [ ]:
w_spectrograms_male = np.array(windowed_spectrograms_male)
w_spectrograms_32_male = w_spectrograms_male.astype('float32')

w_labels_male = np.array(windowed_labels_male, dtype=int)
if dataset == 'EmoDB' or dataset == 'EmoMatch':
  one_hot_labels_emotions_male = np.eye(n_classes)[w_labels_male]
elif dataset == 'RAVDESS':
  one_hot_labels_emotions_male = np.eye(n_classes)[w_labels_male - 1]

np.savez(f'/home/jovyan/Daniel/Exp_{dataset}/datasets/preprocessed/{dataset}100ms32LabeledMaleSpe.npz', x_train = w_spectrograms_32_male, y_emo = one_hot_labels_emotions_male)

In [ ]:
w_spectrograms_female = np.array(windowed_spectrograms_female)
w_spectrograms_32_female = w_spectrograms_female.astype('float32')

w_labels_female = np.array(windowed_labels_female, dtype=int)
if dataset == 'EmoDB' or dataset == 'EmoMatch':
  one_hot_labels_emotions_female = np.eye(n_classes)[w_labels_female]
elif dataset == 'RAVDESS':
  one_hot_labels_emotions_female = np.eye(n_classes)[w_labels_female - 1]

np.savez(f'/home/jovyan/Daniel/Exp_{dataset}/datasets/preprocessed/{dataset}100ms32LabeledFemaleSpe.npz', x_train = w_spectrograms_32_female, y_emo = one_hot_labels_emotions_female)

In [ ]:
for speaker_id in speakers:
    speaker = f"{speaker_id:02d}"

    w_spectrograms = np.array(get_spectrogram_list[speaker])
    w_spectrograms_32 = w_spectrograms.astype('float32')

    windowed_labels = np.array(get_label_list[speaker], dtype=int)
    if dataset == 'EmoDB' or dataset == 'EmoMatch':
      one_hot_labels_emotions = np.eye(n_classes)[windowed_labels]
    elif dataset == 'RAVDESS':
      one_hot_labels_emotions = np.eye(n_classes)[windowed_labels - 1]

    np.savez(f'/home/jovyan/Daniel/Exp_{dataset}/datasets/preprocessed/{dataset}100ms32LabeledSpe{speaker_id}.npz', x_train = w_spectrograms_32, y_emo = one_hot_labels_emotions)

### Evaluate quality of spectrogram-to-audio reconstruction

In [ ]:
num_show = 11

# original audios in  output_audio_path
# spectrograms    in  w_spectrograms_32

def mel_to_audio(mel_spec):
    mel_spec = librosa.db_to_power(mel_spec)
    audio = librosa.feature.inverse.mel_to_audio(mel_spec, sr=sr, hop_length=hop_length)
    return audio

In [ ]:
output_audio_files = os.listdir(output_audio_path)

for audio_file in output_audio_files[:num_show]:
    audio_path = os.path.join(output_audio_path, audio_file)
    audio, _ = librosa.load(audio_path, sr=sr)
    print(f"Playing {audio_file}:")
    ipd.display(Audio(audio, rate=sr))

Playing 41_Ira_12.WAV_segment_0.wav:


Playing 41_Ira_12.WAV_segment_1.wav:


Playing 41_Ira_12.WAV_segment_2.wav:


Playing 37_Ira_06.WAV_segment_0.wav:


Playing 37_Ira_06.WAV_segment_1.wav:


...

Playing 37_Ira_06.WAV_segment_3.wav:


Playing 37_Ira_06.WAV_segment_4.wav:


Playing 37_Ira_06.WAV_segment_5.wav:


Playing 37_Ira_06.WAV_segment_6.wav:


Playing 37_Ira_06.WAV_segment_7.wav:


In [ ]:
for i, mel_spec in enumerate(w_spectrograms_32[:num_show]):
    audio = mel_to_audio(mel_spec)
    filename = f'audio_{i}.wav'
    wavfile.write(filename, sr, audio)
    display(Audio(filename))